<h4>Overview</h4>

This notebook will be the initial data cleaning of the E-Commerce Data dataset found in the data folder file. This dataset contains all transactions that occured between 01/12/2010 and 09/12/2011 for a UK-based and registered non-store online retail. This dataset has 8 features and over 540,000 transactions. The company mainly sells unique all-occasion gifts. Many customers of the company are wholesalers.

Link to the Kaggle dataset webpage: https://www.kaggle.com/carrie1/ecommerce-data <br>
This dataset was extracted from this webpage: http://archive.ics.uci.edu/ml/datasets/Online+Retail

<h4>Objective</h4>

There are two objectives (1) clean these datasets of irrelevant transactions (transactions with too many missing values, and major outliers), and (2) subset the original dataset for future data analysis to find trends and determine the type of customer that brings the highest value.


Questions that may be answered through data exploration:
<ul>
    <li>What is the effect of price and number of purchases</li>
    <li>Are there patterns associated with country?</li>
    <li>Are there certain products which are significantly more popular than others? Could time of year, or country serve as early indicators?</li>
    
    
 </ul>

In [1]:
#libraries, packages
import pandas as pd
import datetime as dt
import matplotlib.pyplot as plt

In [2]:
fileName = "../data/raw/uk_retail.csv"

colTypes = {'InvoiceNo': str, 'StockCode': str, 'Description': str, 'Quantity': int, 'InvoiceDate': str, 'UnitPrice': float, 'CustomerID': object, 'Country':str}
colDates = ["InvoiceDate"]
df = pd.read_csv(fileName, header=0, encoding='unicode_escape', dtype=colTypes, parse_dates=colDates)


In [3]:
df.dtypes

InvoiceNo              object
StockCode              object
Description            object
Quantity                int32
InvoiceDate    datetime64[ns]
UnitPrice             float64
CustomerID             object
Country                object
dtype: object

When reading the data into a dataframe, the "InvoiceDate" column is parsed as a datetime data type. The "Unit Price" column is parsed as a float. The remaining columns are parsed as string/object data type.

In [4]:
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom


A description of the features was provided along with this dataset. It is summarized here.

<table>
    <tr>
        <th>Feature</th>
        <th>Description</th>
    </tr>
    <tr>
        <td>InvoiceNo</td> 
        <td>Invoice number. Nominal, a 6-digit integral number uniquely assigned to each transaction. If this code starts with letter ‘C’, it indicates a cancellation. </td>
    </tr>
    <tr>
        <td>StockCode</td>
        <td>Product (item) code. Nominal, a 5-digit integral number uniquely assigned to each distinct product.</td>
    </tr>
    <tr>
        <td>Description</td>
        <td>Product (item) name. Nominal.</td>
    </tr>
    <tr>
        <td>Quantity</td>
        <td>The quantities of each product  (item) per transaction. Numeric.</td>
    </tr>
    <tr>
        <td>InvoiceDate</td>
        <td>Invoice Date and time. Numeric, the day and time when each transaction was generated.</td>
    </tr>
    <tr>
        <td>UnitPrice</td>
        <td>Unit price. Numeric, Product price per unit in sterling.</td>
    </tr>
    <tr>
        <td>CustomerID</td>
        <td>Customer number. Nominal, a 5-digit integral number uniquely assigned to each customer.</td>
    </tr>
    <tr>
        <td>Country</td>
        <td>Country name. Nominal, the name of the country where each customer resides.</td>
    </tr>
</table>

It's important to note that the invoice number may have a leading a special code character, "c", which represents cancelled orders. We must determine how to deal with cancelled orders, but cancelled transactions shouldn't just be removed as they could offer insight.

<h3>First look at the dataset</h3>

Let's begin with taking a general view of the dataset. 

In [5]:
df.describe()

,Quantity,UnitPrice
count,541909.000000,541909.000000
mean,9.552250,4.611114
std,218.081158,96.759853
min,-80995.000000,-11062.060000
25%,1.000000,1.250000
50%,3.000000,2.080000
75%,10.000000,4.130000
max,80995.000000,38970.000000


In [6]:
df.shape

(541909, 8)

From a short overview of the data we see there are 541,909 transactions. There appears to be negative values for quantity and unit price. Negative values could represent refunds but the dataset didn't provide explanations for negative values. Without knowing what negative values represent, the best decision may be to remove these transactions for further analysis. They may also be outliers in our dataset and therefore may be removed in order to have a more practical prediction model. Transactions with negative numbers may also be reversed transactions so it may be worth while to find if duplicate InvoiceNo exists.

<h3>Looking for missing values, duplicates, and outliers</h3>

The next step will be to address missing values, duplicates values, and outliers. 

In [7]:
df.isnull().sum()

InvoiceNo           0
StockCode           0
Description      1454
Quantity            0
InvoiceDate         0
UnitPrice           0
CustomerID     135080
Country             0
dtype: int64

Checking for missing values. It appears only two features contain any missing values: <i>Description</i>, and <i>CustomerID</i>. The percentage of transactions that have any missing values represents about 25% of the total dataset. It a decent portion of the dataset but even if we remove the transactions with any missing data we are still left with over 400,000 elements. 

In [8]:
df[df['CustomerID'].isnull()]

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
622,536414,22139,NaN,56,2010-12-01 11:52:00,0.00,NaN,United Kingdom
1443,536544,21773,DECORATIVE ROSE BATHROOM BOTTLE,1,2010-12-01 14:32:00,2.51,NaN,United Kingdom
1444,536544,21774,DECORATIVE CATS BATHROOM BOTTLE,2,2010-12-01 14:32:00,2.51,NaN,United Kingdom
1445,536544,21786,POLKADOT RAIN HAT,4,2010-12-01 14:32:00,0.85,NaN,United Kingdom
1446,536544,21787,RAIN PONCHO RETROSPOT,2,2010-12-01 14:32:00,1.66,NaN,United Kingdom
...,...,...,...,...,...,...,...,...
541536,581498,85099B,JUMBO BAG RED RETROSPOT,5,2011-12-09 10:26:00,4.13,NaN,United Kingdom
541537,581498,85099C,JUMBO BAG BAROQUE BLACK WHITE,4,2011-12-09 10:26:00,4.13,NaN,United Kingdom
541538,581498,85150,LADIES & GENTLEMEN METAL SIGN,1,2011-12-09 10:26:00,4.96,NaN,United Kingdom
541539,581498,85174,S/4 CACTI CANDLES,1,2011-12-09 10:26:00,10.79,NaN,United Kingdom


Looking at a sample of the transactions that have missing CustomerID values, it seems like we won't be able to use those transactions when trying to identify trends based on customers. Though we can still use those transactions to explore trends grouped by Country or product popularity. So we can subset a dataframe that contains the transactions with any missing data.

In [9]:
nullDf = df[(df['CustomerID'].isnull()) | (df['Description'].isnull())] #Dataeframe with transactions with any null values.
nullDf.shape

(135080, 8)

Now we move on to look for any cancelled orders.

In [10]:
df['InvoiceNo'].str.startswith('C').value_counts()

False    532621
True       9288
Name: InvoiceNo, dtype: int64

In the raw dataset, there were 9,288 cancelled orders. We'll create a subset of the data set which contains only cancelled transactions. Next, we'll look at transactions with irregular values, like negative prices or quantities.

In [11]:
cancelledDf = df[df['InvoiceNo'].str.startswith('C')] #dataframe which contains the cancelled transactions.
cancelledDf.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
141,C536379,D,Discount,-1,2010-12-01 09:41:00,27.50,14527,United Kingdom
154,C536383,35004C,SET OF 3 COLOURED FLYING DUCKS,-1,2010-12-01 09:49:00,4.65,15311,United Kingdom
235,C536391,22556,PLASTERS IN TIN CIRCUS PARADE,-12,2010-12-01 10:24:00,1.65,17548,United Kingdom
236,C536391,21984,PACK OF 12 PINK PAISLEY TISSUES,-24,2010-12-01 10:24:00,0.29,17548,United Kingdom
237,C536391,21983,PACK OF 12 BLUE PAISLEY TISSUES,-24,2010-12-01 10:24:00,0.29,17548,United Kingdom


The cancelled orders have negative quantity values so there may an associated transaction which may need to be excluded.

In [12]:
cancelledInvoices = cancelledDf['InvoiceNo'].str.strip("C")
ordersCancelled = df[df['InvoiceNo'].isin(cancelledInvoices)]
ordersCancelled.shape

(0, 8)

In [13]:
#an additional sanity check
df[df['InvoiceNo']=='536383']

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country


Cancelled orders do not reverse another transaction. We'll exclude the cancelled transactions when modeling for value but we'll have the cancelled orders dataframe for cancelled orderd analysis.

In [14]:
df[df['UnitPrice']<0]

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
299983,A563186,B,Adjust bad debt,1,2011-08-12 14:51:00,-11062.06,NaN,United Kingdom
299984,A563187,B,Adjust bad debt,1,2011-08-12 14:52:00,-11062.06,NaN,United Kingdom


From examining negative unit prices, we see that the dataset contains more than just customer transactions. It contains internal business accounting transactions.

In [15]:
df[df['InvoiceNo'].str.startswith('A')]

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
299982,A563185,B,Adjust bad debt,1,2011-08-12 14:50:00,11062.06,NaN,United Kingdom
299983,A563186,B,Adjust bad debt,1,2011-08-12 14:51:00,-11062.06,NaN,United Kingdom
299984,A563187,B,Adjust bad debt,1,2011-08-12 14:52:00,-11062.06,NaN,United Kingdom


Instead of looking at negative unit price values, we can look at the InvoiceNo and StockCode to find more special transaction which were not provided in the dataset description.

In [16]:
specialStockCodeDf = df[df['StockCode'].str.isalpha()] #This dataframe contains the transactions with non-customer related stock codes.
specialStockCodeDf.shape

(2759, 8)

Below is a list of special stock codes.

In [17]:
specialStockCodeList = df[df['StockCode'].str.isalpha()]['StockCode'].unique()
print(specialStockCodeList)

['POST' 'D' 'DOT' 'M' 'S' 'AMAZONFEE' 'm' 'DCGSSBOY' 'DCGSSGIRL' 'PADS'
 'B' 'CRUK']


Right below, we take a sample from the dataset of each special stock code.

In [18]:
sample = pd.DataFrame(columns=colTypes, )
for code in specialStockCodeList:
    sample = sample.append(specialStockCodeDf[specialStockCodeDf['StockCode']==code].head(1))
sample

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
45,536370,POST,POSTAGE,3,2010-12-01 08:45:00,18.000,12583,France
141,C536379,D,Discount,-1,2010-12-01 09:41:00,27.500,14527,United Kingdom
1814,536544,DOT,DOTCOM POSTAGE,1,2010-12-01 14:32:00,569.770,NaN,United Kingdom
2239,536569,M,Manual,1,2010-12-01 15:35:00,1.250,16274,United Kingdom
14436,C537581,S,SAMPLES,-1,2010-12-07 12:03:00,12.950,NaN,United Kingdom
14514,C537600,AMAZONFEE,AMAZON FEE,-1,2010-12-07 12:41:00,1.000,NaN,United Kingdom
40383,539736,m,Manual,1,2010-12-21 15:18:00,2.550,NaN,United Kingdom
84016,543358,DCGSSBOY,BOYS PARTY BAG,1,2011-02-07 14:04:00,3.290,NaN,United Kingdom
84017,543358,DCGSSGIRL,GIRLS PARTY BAG,3,2011-02-07 14:04:00,3.290,NaN,United Kingdom
157195,550193,PADS,PADS TO MATCH ALL CUSHIONS,1,2011-04-15 09:27:00,0.001,13952,United Kingdom


It appears the special stock codes mostly relate to internal business transactions, not customer purchases. For this reason, they will be removed for future data analysis.

In [19]:
#create the final subset of data which excludes transactions with any missing values, cancelled orders, and special stock codes.

cleanedDf = df[(~df['InvoiceNo'].isin(specialStockCodeDf['InvoiceNo'])) & (~df['InvoiceNo'].isin(cancelledDf['InvoiceNo'])) & (~df['InvoiceNo'].isin(nullDf['InvoiceNo']))]
cleanedDf

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom
...,...,...,...,...,...,...,...,...
541904,581587,22613,PACK OF 20 SPACEBOY NAPKINS,12,2011-12-09 12:50:00,0.85,12680,France
541905,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,2011-12-09 12:50:00,2.10,12680,France
541906,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,2011-12-09 12:50:00,4.15,12680,France
541907,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,2011-12-09 12:50:00,4.15,12680,France


In [20]:
cleanedDf.sort_values(by='UnitPrice', ascending=False).head(5)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
222682,556446,22502,PICNIC BASKET WICKER 60 PIECES,1,2011-06-10 15:33:00,649.5,15098,United Kingdom
222680,556444,22502,PICNIC BASKET WICKER 60 PIECES,60,2011-06-10 15:28:00,649.5,15098,United Kingdom
205759,554836,22655,VINTAGE RED KITCHEN CABINET,1,2011-05-26 16:25:00,295.0,13015,United Kingdom
32484,539080,22655,VINTAGE RED KITCHEN CABINET,1,2010-12-16 08:41:00,295.0,16607,United Kingdom
133994,547814,22656,VINTAGE BLUE KITCHEN CABINET,1,2011-03-25 14:19:00,295.0,13452,United Kingdom


In [21]:
cleanedDf.describe()

,Quantity,UnitPrice
count,360166.000000,360166.000000
mean,13.388529,2.886688
std,189.510089,4.466709
min,1.000000,0.000000
25%,2.000000,1.250000
50%,5.000000,1.950000
75%,12.000000,3.750000
max,80995.000000,649.500000


In summary we have the following dataframes for future analysis.

<table>
    <tr>
        <th>Dataset</th>
        <th>Description</th>
    </tr>
    <tr>
        <td>df</td>
        <td>The raw dataset</td>
    </tr>
    <tr>
        <td>nullDf</td>
        <td>Contains any transaction with a null value</td>
    </tr>
    <tr>
        <td>cancelledDf</td>
        <td>Contains the cancelled order transactions</td>
    </tr>
    <tr>
        <td>specialStockCodeDf</td>
        <td>Contains non-customer related transactions</td>
    </tr>
    <tr>
        <td>cleanedDf</td>
        <td>Dataframe that excludes transactions found in nullDf, cancelledDf, and specialStockCodeDf.<br> Intention is this dataframe only contains customer transactions</td>
    </tr>
</table>
    

In [22]:
cleanedDf.to_csv("../data/processed/processedUkData.csv")